<a href="https://colab.research.google.com/github/singhsukhendra/Mytest/blob/master/cvnn_on_wdbc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# simple neural network of one complex valued neuron
# extended to use a periodic activation function
import numpy
# pandas for reading csv files
import pandas
# sklearn for splitting data into test/train 
import sklearn.model_selection

In [0]:
class neuralNetwork:
    
    def __init__(self, inputs, cats, periods):
        # number of inputs
        self.inputs = inputs
        
        # link weights matrix
        self.w = numpy.random.normal(0.0, pow(1.0, -0.5), (self.inputs + 1))
        self.w = numpy.array(self.w, ndmin=2, dtype='complex128')
        self.w += 1j * numpy.random.normal(0.0, pow(1.0, -0.5), (self.inputs + 1))
        
        # testing overrride
        #self.w = numpy.array([1.0 + 0.0j, 1.0 + 0.0j], ndmin=2, dtype='complex128')
        
        # number of output class categories
        self.categories = cats
        
        # todo periodicity
        self.periodicity = periods
        
        pass
    
    def z_to_class(self, z):
        # first work out the angle, but shift angle from [-pi/2, +pi.2] to [0,2pi]
        angle = numpy.mod(numpy.angle(z) + 2*numpy.pi, 2*numpy.pi)
        # from angle to category
        p = int(numpy.floor (self.categories * self.periodicity * angle / (2*numpy.pi)))
        p = numpy.mod(p, self.categories)
        return p

    def class_to_angles(self, c):
        # class to several angles due to periodicity, using bisector
        angles = (c + 0.5 + (self.categories * numpy.arange(self.periodicity))) / (self.categories * self.periodicity) * 2 * numpy.pi
        return angles
    
    def status(self):
        print ("w = ", self.w)
        print ("categories = ", self.categories)
        print ("periodicity = ", self.periodicity)
        pass

    def query(self, inputs_list):
        # add bias input
        inputs_list.append(1.0)
        
        # convert input to complex
        inputs = numpy.array(inputs_list, ndmin=2, dtype='complex128').T
        #print("inputs = \n", inputs)
        
        # combine inputs, weighted
        z = numpy.dot(self.w, inputs)
        #print("z = ", z)
        
        # map to output classes
        o = self.z_to_class(z)
        #print("output = ", o)
        #print ("")
        return o
    
    def train(self, inputs_list, target):
        # add bias input
        inputs_list.append(1.0)
        
        # convert inputs and outputs list to 2d array
        inputs = numpy.array(inputs_list, ndmin=2, dtype='complex128').T

        # combine inputs, weighted
        z = numpy.dot(self.w, inputs)[0]
        
        # desired angle from trainging set
        # first get all possible angles
        desired_angles = self.class_to_angles(target)
        
        # potential errors errors
        errors =  numpy.exp(1j*desired_angles) - z
        # select smallest error
        e = errors[numpy.argmin(numpy.abs(errors))]
        
        # dw = e * x.T / (x.x.T)
        dw = (e * numpy.conj(inputs.T)) / (self.inputs + 1)
        #print("dw = ", dw)
        self.w += dw
        #print("new self.w = ", self.w )
        #print("test new self.w with query = ", self.query(inputs.T))
        #print("--")
    pass


In [15]:
# create instance of neural network
number_of_inputs = 30
categories = 3
periods = 3

n = neuralNetwork(number_of_inputs, categories, periods)
n.status()

w =  [[ 0.17204416+0.61684602j  0.20817702-0.64850371j  0.70740581-0.27249121j
   0.41656897+0.36225549j  0.67733625+0.35681202j  0.5529395 -2.49229864j
   0.90138143-1.64373125j  0.53883932+0.29478896j -0.70627641+0.49530829j
  -0.26695782+0.2599042j  -0.76519984+0.56732867j  0.60811906+0.67982317j
   0.44715657+0.89179868j -0.4110119 +0.10882038j  1.23436185-0.87576174j
   0.5999259 +1.02196114j -0.24220376-0.9480309j  -2.8424802 +1.36396224j
   0.51049141+0.43604909j -0.39787047+1.04209732j  1.67626554+1.47345094j
  -0.96370746-1.77540428j  0.94773979-0.04594501j  0.88581579+1.07932438j
  -0.04621717-0.01713838j -0.28756343-1.26470685j -1.29130487+0.03722388j
  -2.68873007+0.84684137j -0.21274738+2.1681661j  -0.57156294+1.30549098j
  -1.76391291+0.62652555j]]
categories =  3
periodicity =  3


In [16]:
# load the iris training data CSV file into a list
df = pandas.read_csv('data1.csv')

# scale the lengths
df[['radius_mean','texture_mean','perimeter_mean','area_mean','smoothness_mean','compactness_mean',	'concavity_mean','concave_points_mean','symmetry_mean','fractal_dimension_mean','radius_se','texture_se','perimeter_se',
'area_se','smoothness_se','compactness_se','concavity_se','concave_points_se','symmetry_se','fractal_dimension_se',
'radius_worst','texture_worst','perimeter_worst','area_worst','smoothness_worst','compactness_worst','concavity_worst',
'concave_points_worst','symmetry_worst','fractal_dimension_worst']] = df[['radius_mean','texture_mean','perimeter_mean','area_mean','smoothness_mean','compactness_mean',	'concavity_mean','concave_points_mean','symmetry_mean','fractal_dimension_mean','radius_se','texture_se','perimeter_se',
'area_se','smoothness_se','compactness_se','concavity_se','concave_points_se','symmetry_se','fractal_dimension_se',
'radius_worst','texture_worst','perimeter_worst','area_worst','smoothness_worst','compactness_worst','concavity_worst',
'concave_points_worst','symmetry_worst','fractal_dimension_worst']].astype(numpy.float64) * 0.01

# shuffle and split dataframe into train and test sets, split 3/4 and 1/4
iris_train, iris_test = sklearn.model_selection.train_test_split(df, train_size=0.80)
iris_test.head()

,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave_points_mean,symmetry_mean,fractal_dimension_mean,radius_se,texture_se,perimeter_se,area_se,smoothness_se,compactness_se,concavity_se,concave_points_se,symmetry_se,fractal_dimension_se,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave_points_worst,symmetry_worst,fractal_dimension_worst,diagnosis
451,0.1959,0.2500,1.2770,11.910,0.001032,0.000987,0.001655,0.000906,0.001663,0.000539,0.004674,0.013750,0.02916,0.5618,0.000119,0.000193,0.000491,0.000150,0.000164,0.000018,0.2144,0.3096,1.3980,14.210,0.001528,0.001845,0.003977,0.001466,0.002293,0.000609,1
266,0.1060,0.1895,0.6928,3.464,0.000969,0.001147,0.000639,0.000264,0.001922,0.000649,0.004505,0.011970,0.03430,0.2710,0.000075,0.000358,0.000335,0.000137,0.000350,0.000033,0.1188,0.2294,0.7828,4.248,0.001213,0.002515,0.001916,0.000793,0.002940,0.000759,0
167,0.1678,0.1880,1.0930,8.863,0.000887,0.000918,0.000842,0.000658,0.001893,0.000553,0.005990,0.013910,0.04129,0.6734,0.000061,0.000247,0.000263,0.000160,0.000209,0.000035,0.2005,0.2630,1.3070,12.600,0.001168,0.002119,0.002318,0.001474,0.002810,0.000723,1
44,0.1317,0.2181,0.8542,5.315,0.000971,0.001047,0.000826,0.000525,0.001746,0.000618,0.001938,0.006123,0.01334,0.1449,0.000034,0.000138,0.000145,0.000069,0.000111,0.000017,0.1623,0.2989,1.0550,7.407,0.001503,0.003904,0.003728,0.001607,0.003693,0.000962,1
544,0.1387,0.2070,0.8977,5.848,0.000958,0.001018,0.000369,0.000237,0.001620,0.000669,0.002720,0.010470,0.02076,0.2312,0.000063,0.000217,0.000262,0.000091,0.000149,0.000036,0.1505,0.2475,0.9917,6.886,0.001264,0.002037,0.001377,0.000684,0.002249,0.000849,0


In [0]:
# train neural network

epochs = 2

for e in range(epochs):
    # go through all records in the training data set
    for idx, data in iris_train.iterrows():
        data_list = data.tolist()
        species = data_list[0]
        lengths = data_list[1:]
        n.train(lengths, species)
        pass
    pass

In [18]:
n.status()

w =  [[-9.21119156e+198+9.41647274e+198j -2.10441401e+199+2.15131311e+199j
  -1.01418962e+200+1.03679191e+200j -2.28258207e+196+2.33345184e+196j
  -1.34632163e+196+1.37632584e+196j -1.08240542e+196+1.10652798e+196j
  -2.34695528e+195+2.39925967e+195j -4.31475009e+196+4.41090887e+196j
  -1.70763293e+196+1.74568935e+196j -2.34190176e+196+2.39409353e+196j
  -4.04025605e+197+4.13029745e+197j -1.72060135e+197+1.75894677e+197j
   1.44506682e+198-1.47727167e+198j -2.70086399e+195+2.76105561e+195j
  -6.82130929e+195+6.97332941e+195j -1.18182606e+196+1.20816431e+196j
  -2.10254772e+195+2.14940523e+195j -4.38089024e+195+4.47852303e+195j
  -8.00799111e+194+8.18645769e+194j -3.39151481e+198+3.46709831e+198j
  -1.10992886e+199+1.13466480e+199j -2.14297385e+199+2.19073230e+199j
  -9.14275586e+199+9.34651188e+199j -3.44756083e+196+3.52439338e+196j
  -4.20036369e+196+4.29397326e+196j -6.02305317e+196+6.15728330e+196j
  -1.16549317e+196+1.19146742e+196j -6.33508694e+196+6.47627108e+196j
  -2.10480071e+

In [19]:
# query after training

scorecard = []

for idx, data in iris_test.iterrows():
    data_list = data.tolist()
    correct_species = int(data_list[0])
    lengths = data_list[1:]
    answer = n.query(lengths)
    print(correct_species, answer)
    if (answer == correct_species):
        # network's answer matches correct answer, add 1 to scorecard
        scorecard.append(1)
    else:
        # network's answer doesn't match correct answer, add 0 to scorecard
        scorecard.append(0)
        pass
    pass

0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0


In [20]:
# calculate the performance score, the fraction of correct answers
scorecard_array = numpy.asarray(scorecard)
print ("performance = ", scorecard_array.sum() / scorecard_array.size)

performance =  1.0
